In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from hda import Client
import os
import datetime
import time
import matplotlib
import cfgrib

In [2]:
c = Client(debug=True)
month = "0" + str(1)
print(month)

01


In [3]:
data = {
  "datasetId": "EO:ECMWF:DAT:REANALYSIS_ERA5_LAND",
  "boundingBoxValues": [
    {
      "name": "area",
      "bbox": [
        4.919044320039164,
        45.73676238441797,
        4.919044320039165,
        45.73676238441798,
      ]
    }
  ],
  "multiStringSelectValues": [
    {
      "name": "day",
      "value": [
        "10",
        "11",
        "12",
        "13",
        "14",
        "15",
        "16",
        "17",
        "18",
        "19",
        "20",
        "21",
        "22",
        "23",
        "24",
        "25",
        "26",
        "27",
        "28",
        "29",
        "30",
        "31",
        "01",
        "02",
        "03",
        "04",
        "05",
        "06",
        "07",
        "08",
        "09"
      ]
    },
    {
      "name": "time",
      "value": [
        "00:00",
        "01:00",
        "02:00",
        "03:00",
        "04:00",
        "05:00",
        "06:00",
        "07:00",
        "08:00",
        "09:00",
        "10:00",
        "11:00",
        "12:00",
        "13:00",
        "14:00",
        "15:00",
        "16:00",
        "17:00",
        "18:00",
        "19:00",
        "20:00",
        "21:00",
        "22:00",
        "23:00"
      ]
    },
    {
      "name": "variable",
      "value": [
        "2m_temperature"
      ]
    }
  ],
  "stringChoiceValues": [
    {
      "name": "format",
      "value": "netcdf"
    },
    {
      "name": "month",
      "value": "01"
    },
    {
      "name": "year",
      "value": "2022"
    }
  ]
}

In [4]:
stamp = time.time()
datasets = []

dates = ['01','02','03','04','05','06','07','08','09','10','11','12']

for i in dates:
    data['stringChoiceValues'][1]['value'] = i
    matches = c.search(data)
    matches.download()
    
    for match in matches.results:
        fdst = match['filename']
        print(f"Found: {fdst}")
        era5_ds = xr.open_dataset(fdst)
        xr.decode_cf(era5_ds)
        datasets.append(era5_ds)

#filename = '2m_temperatures_1h_lyon' + str(int(stamp)) + '.nc'

#os.rename(fdst, filename)

Found: adaptor.mars.internal-1686496765.3737743-19693-9-d47ed260-17d3-4dbe-9da7-1c427c4b17d3.nc



  0%|                                                                                      | 0.00/4.99k [00:00<?, ?B/s]
                                                                                                                       

Found: adaptor.mars.internal-1686506323.3676383-2061-15-0cd439fe-cc90-4449-b4ed-ca84bd5725c7.nc


Found: adaptor.mars.internal-1686507753.0149162-7790-16-254d2a06-f699-4b25-bfc7-80f3fb7fa54c.nc



  0%|                                                                                      | 0.00/5.28k [00:00<?, ?B/s]
                                                                                                                       

...


  0%|                                                                                      | 0.00/5.41k [00:00<?, ?B/s]
                                                                                                                       

Found: adaptor.mars.internal-1686516940.612957-28399-17-52405a86-b20d-4b90-9478-e3c96c7e3e10.nc


Found: adaptor.mars.internal-1686517063.472406-15898-14-23946a3c-0579-40c8-891d-8494c661c7c3.nc



  0%|                                                                                      | 0.00/5.41k [00:00<?, ?B/s]
                                                                                                                       

Found: adaptor.mars.internal-1686517177.110005-17559-1-5212a310-cdc1-4626-9221-f5d120347aac.nc


In [5]:
#era5_ds = xr.open_dataset(filename)
#xr.decode_cf(era5_ds)

#era5_ds = xr.open_dataset(filename, engine="cfgrib")

era5_ds = xr.concat(datasets, dim='time')

In [6]:
print(era5_ds)

lats = era5_ds['latitude']
print("Latitude South: {0:.2f}, Latitude North: {1:.2f}".format(lats.data.min(), lats.data.max()))

lons = era5_ds['longitude']
print("Latitude West: {0:.2f}, Latitude East: {1:.2f}".format(lons.data.min(), lons.data.max()))

<xarray.Dataset>
Dimensions:    (longitude: 1, latitude: 1, time: 8760)
Coordinates:
  * longitude  (longitude) float32 4.92
  * latitude   (latitude) float32 45.74
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 277.5 277.2 ... 286.0 285.8
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-11 15:19:26 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...
Latitude South: 45.74, Latitude North: 45.74
Latitude West: 4.92, Latitude East: 4.92


In [7]:
temperature2m = era5_ds['t2m']
print(temperature2m)

<xarray.DataArray 't2m' (time: 8760, latitude: 1, longitude: 1)>
array([[[277.51062]],

       [[277.2057 ]],

       [[276.95145]],

       ...,

       [[286.25412]],

       [[285.95935]],

       [[285.77353]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 4.92
  * latitude   (latitude) float32 45.74
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-12-31T23:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature


In [8]:
print(temperature2m.values)

[[[277.51062]]

 [[277.2057 ]]

 [[276.95145]]

 ...

 [[286.25412]]

 [[285.95935]]

 [[285.77353]]]


In [9]:
print(temperature2m.coords)

Coordinates:
  * longitude  (longitude) float32 4.92
  * latitude   (latitude) float32 45.74
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-12-31T23:00:00


In [10]:
print(temperature2m.coords['longitude'].values)

[4.92]


In [11]:
print(temperature2m.coords['latitude'].values)

[45.736]


In [12]:
print(temperature2m.coords['time'].values)

['2022-01-01T00:00:00.000000000' '2022-01-01T01:00:00.000000000'
 '2022-01-01T02:00:00.000000000' ... '2022-12-31T21:00:00.000000000'
 '2022-12-31T22:00:00.000000000' '2022-12-31T23:00:00.000000000']


In [13]:
print(temperature2m.attrs)

{'units': 'K', 'long_name': '2 metre temperature'}


In [14]:
print(temperature2m.attrs['units'])

K


In [15]:
print(temperature2m.attrs['long_name'])

2 metre temperature


In [16]:
for i in range (0, 1):
    (temperature2m[i, :, :]-273.15).plot(cmap = 'jet')
    time.sleep(1)

TypeError: No numeric data to plot.

In [17]:
temperature2m.mean() - 273.15

<xarray.DataArray 't2m' ()>
array(13.82488403)

In [18]:
(temperature2m.sel(time = '2022-05-16T14:00:00.000000000')-273.15).plot(cmap = 'jet')

TypeError: No numeric data to plot.

In [19]:
temperature2m

<xarray.DataArray 't2m' (time: 8760, latitude: 1, longitude: 1)>
array([[[277.51062]],

       [[277.2057 ]],

       [[276.95145]],

       ...,

       [[286.25412]],

       [[285.95935]],

       [[285.77353]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 4.92
  * latitude   (latitude) float32 45.74
  * time       (time) datetime64[ns] 2022-01-01 ... 2022-12-31T23:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature

In [20]:
"""
finding frost days: 
"""

daily_mean_temps = temperature2m.min(dim=['longitude', 'latitude'])

#print(daily_mean_temps)

daily_temps = daily_mean_temps.resample(time='D').min()    #taking min values of temperature for the region in an array

under0_days = daily_temps.where(daily_temps.values < 273.15)    #extract array with days where min temp was under 0
under0_days = under0_days.dropna(dim='time')

above2_days = daily_temps.where(daily_temps.values > 275)      #extract array with days where min temp was above 2
above2_days = above2_days.dropna(dim = 'time')

#print(daily_temps)
#print(under0_days)

frost_array = []
for i in under0_days.time:
    frost_array.append(i.values)       #append the time values of the days where there was frost


In [21]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']



frost_first_half = [date for date in frost_array if (pd.to_datetime(date).month) < 7]     #cut year in 2 bf4 july to find seas
frost_2nd_half = [date for date in frost_array if (pd.to_datetime(date).month) >= 7]     #cut year in 2 after july


first_frost_day = pd.to_datetime(min(frost_2nd_half))
last_frost_day = pd.to_datetime(max(frost_first_half))

#do the same for above2_days




#print(first_frost_day)
#print(last_frost_day)

print(f"In {first_frost_day.year}, the last frost day happened on {last_frost_day}, and frost came back on {first_frost_day}. The growing season of this place is hence between {months[last_frost_day.month - 1]} and {months[first_frost_day.month - 1]}.")

In 2022, the last frost day happened on 2022-04-04 00:00:00, and frost came back on 2022-12-06 00:00:00. The growing season of this place is hence between April and December.


In [22]:
"""
Calculate median temperature over growing season:
"""
print(last_frost_day, first_frost_day, type(last_frost_day))

last_frost_day = last_frost_day + datetime.timedelta(days=1)
first_frost_day = first_frost_day - datetime.timedelta(days=1)

last_frost_day = datetime.datetime(last_frost_day.year, last_frost_day.month, last_frost_day.day)
first_frost_day = datetime.datetime(first_frost_day.year, first_frost_day.month, first_frost_day.day)


las_tmsp = pd.to_datetime(last_frost_day)
first_tmsp = pd.to_datetime(first_frost_day)

print(las_tmsp, first_tmsp)



mask = (temperature2m.time > las_tmsp) & (temperature2m.time < first_tmsp)

seasons_temps = temperature2m.sel(time=mask)      #array with temperatures of the growing season

seasons_temps

2022-04-04 00:00:00 2022-12-06 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2022-04-05 00:00:00 2022-12-05 00:00:00


<xarray.DataArray 't2m' (time: 5855, latitude: 1, longitude: 1)>
array([[[274.95746]],

       [[274.878  ]],

       [[274.36096]],

       ...,

       [[275.96667]],

       [[275.3935 ]],

       [[274.9991 ]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 4.92
  * latitude   (latitude) float32 45.74
  * time       (time) datetime64[ns] 2022-04-05T01:00:00 ... 2022-12-04T23:00:00
Attributes:
    units:      K
    long_name:  2 metre temperature

In [23]:
print(seasons_temps.values.min()-273.15)

0.03652343750002274


In [24]:
#create array with median daily temperatures

daily_mean_temps = seasons_temps.mean(dim=['longitude', 'latitude']) - 273.15

# Resample the daily mean temperatures to a daily frequency
#daily_temps = daily_mean_temps.resample(time='D').mean()
daily_min_temps = daily_mean_temps.resample(time='D').min()
daily_max_temps = daily_mean_temps.resample(time='D').max()

avg_min = daily_min_temps.mean().values.item()
avg_max = daily_max_temps.mean().values.item()

print(daily_max_temps)

print(avg_min, avg_max)
#print(daily_temps.sel(time='2022-04-05'))
#print(daily_temps.min(), daily_temps.max())

<xarray.DataArray 't2m' (time: 244)>
array([12.968445 , 14.406067 , 11.649078 , 16.00595  , 10.164154 ,
       13.224945 , 16.6427   , 17.813507 , 20.224945 , 20.131805 ,
       20.497314 , 20.026123 , 17.56958  , 18.856415 , 19.148376 ,
       18.606598 , 19.522034 , 20.174316 , 16.539612 , 14.593079 ,
       15.548035 , 17.489777 , 18.785461 , 21.561676 , 21.234985 ,
       17.793182 , 17.74289  , 19.01184  , 19.03894  , 21.20517  ,
       16.961487 , 17.76883  , 20.15741  , 21.426361 , 22.418732 ,
       25.154999 , 27.138947 , 26.689117 , 23.088928 , 25.857025 ,
       27.693665 , 26.11554  , 28.585052 , 30.26416  , 30.035889 ,
       30.339203 , 29.791931 , 32.224518 , 26.33789  , 20.818207 ,
       20.507843 , 22.543243 , 23.639343 , 21.665619 , 20.108704 ,
       21.298676 , 22.26471  , 25.00064  , 28.331604 , 29.070496 ,
       30.99289  , 26.748901 , 24.931458 , 23.223602 , 20.572845 ,
       19.295258 , 23.866486 , 28.3219   , 31.155731 , 26.459747 ,
       29.993988 , 34.646

In [25]:
min_temp_season = daily_min_temps.min().values.item()
max_temp_season = daily_max_temps.max().values.item()

print(min_temp_season, max_temp_season)

0.036529541015625 36.934478759765625
